# Introduction to pandas

* **pandas** is a Python package providing convenient data structures to work with labelled data.

* **pandas** is perfectly suited for observational / statistical data sets, having many similarities with Excel spreadsheets.

* Key features:

    - easy handling of **missing data**
    - **size mutability**: columns can be inserted and deleted from DataFrame
    - automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the user can simply ignore the labels and let Series, DataFrame, etc. automatically align the data for you in computations
    - powerful, flexible **group by** functionality to perform split-apply-combine operations on data sets
    - make it **easy to convert** ragged, differently-indexed data in other Python and NumPy data structures into DataFrame objects
    - intelligent label-based **slicing**, **fancy indexing**, and **subsetting** of large data sets
    - intuitive **merging** and **joining** data sets
    - flexible **reshaping** and pivoting of data sets
    - **hierarchical** labeling of axes (possible to have multiple labels per tick)
    - robust IO tools for loading data from **flat files** (CSV and delimited), Excel files, databases, and saving / loading data from the ultrafast HDF5 format
    - **time series**-specific functionality

### Primary data structures of pandas
* **Series** (1-dimensional)
* **DataFrame** (2-dimensional)

pandas is built on top of **NumPy** and is intended to integrate well within a scientific computing environment with many other 3rd party libraries.

## Loading data

* In this part, we will use observations of the air quality as an example
* The data have been collected **hourly** from one of the monitoring sites in London (Marleybone Road) **over the year 2015**
* Measured are: ozone ($O_3$), nitrogen oxides (NOx), carbon monoxide (CO) and PM10 particulate matter

* Data source: https://uk-air.defra.gov.uk/data/
* Site description: https://uk-air.defra.gov.uk/networks/site-info?uka_id=UKA00315

First, we import pandas module. We use an alias "pd" to write code quicker.

In [ ]:
import pandas as pd

We also import the `os` module that is useful for building paths to files (among many other things). And `numpy` with `matplotlib` just in case too.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fname = '../data/air_quality_hourly_london_marylebone.csv'

Let's try to read the data using `pandas.read_csv()` function.

In [ ]:
# minimal setup to read the given file
air_quality = pd.read_csv(fname, header=4, skipfooter=4, na_values='No data', engine='python')

**Q**: What happens if you remove the header? skipfooter? engine?

## Data structures: `DataFrame` and `Series`

Let's interrogate the `DataFrame` object!

In [ ]:
type(air_quality)

In [ ]:
# Internal nature of the object
print(air_quality.shape)
print()
print(air_quality.dtypes)

In [ ]:
# View just the tip of data
air_quality.head(5)

**Q:** What did you notice about "Status" columns? Compare them to the original text file.

In [ ]:
# View the last rows of data
air_quality.tail(n=2)  # Note the optional argument (available for head() too)

Get descriptors for the **vertical** axis (axis=0):

In [ ]:
air_quality.index

Get descriptors for the horizontal axis (axis=1):

In [ ]:
air_quality.columns

A lot of information at once including memory usage:

In [ ]:
air_quality.info()

### Series, pandas' 1D data containter

A series can be constructed with the `pd.Series` constructor (passing an array of values) or from a `DataFrame`, by extracting one of its columns.

In [ ]:
carbon_monoxide = air_quality['Carbon monoxide']

Some of its attributes:

In [ ]:
print(type(carbon_monoxide))
print(carbon_monoxide.dtype)
print(carbon_monoxide.shape)
print(carbon_monoxide.nbytes)

Show me what you got!

In [ ]:
# uncomment to see the values
# carbon_monoxide

### Numpy as pandas's backend

It is always possible to fall back to a good old NumPy array to pass on to scientific libraries that need them: SciPy, scikit-learn, etc

In [ ]:
air_quality['Nitrogen oxides as nitrogen dioxide'].values

In [ ]:
type(air_quality['Nitrogen oxides as nitrogen dioxide'].values)

## Cleaning data

<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">The truth about data science: cleaning your data is 90% of the work. Fitting the model is easy. Interpreting the results is the other 90%.</p>&mdash; Jake VanderPlas (@jakevdp) <a href="https://twitter.com/jakevdp/status/742406386525446144">June 13, 2016</a></blockquote>
<script async src="//platform.twitter.com/widgets.js" charset="utf-8"></script>

### Dealing with dates and times

In [ ]:
# def dateparse(date_str, time_str):
#     diff = pd.to_timedelta((df['End Date'] == '24:00:00').astype(int), unit='d')
#     pd.datetime.strptime(x+y, '%Y-%m-%d%H:%M:%S') 

In [ ]:
air_quality = pd.read_csv(fname, header=4, skipfooter=4, na_values='No data', engine='python',
                          parse_dates={'Time': [0, 1]})

### Renaming columns

In [ ]:
air_quality.columns = ['Time', 'O3', 'O3_status', 'NOx', 'NOx_status',
                       'CO', 'CO_status', 'PM10', 'PM10_status', 'Co', 'Co_status']
air_quality.columns

### Deleting columns

Let us concentrate our attention on the first 4 chemical species, and remove cobalt data from our `DataFrame`:

In [ ]:
air_quality = air_quality.drop('Co', 1)
air_quality = air_quality.drop('Co_status', 1)

In [ ]:
air_quality.head()

## Basic visualisation

### Exercise

Try calling `plot()` method of the `air_quality` object:

In [ ]:
# air_quality.plot()

What happens if put `subplots=True` as an argument of the `plot()` method?

In [ ]:
# air_quality.plot( ... )

It is easy to create other useful plots using `DataFrame`:

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2)
air_quality.boxplot(ax=ax0, column=['O3', 'PM10'])
air_quality.O3.plot(ax=ax1, kind="kde")

**Q:** the plot is too small. How to make it bigger?

As well as just a simple line plot:

In [ ]:
air_quality.O3.plot(grid=True, figsize=(12, 2))

### Setting missing values

As you may notice, we have negative values of ozone concentration, which does not make sense. So, let us replace those negative values with NaN:

**Q:** how to list data entries with negative O3?

In [ ]:
# your code here

We can mask them out in the same way as you do with numpy arrays:

In [ ]:
# Replace negative ozone values with NaN
air_quality.O3[air_quality.O3.values < 0] = np.nan

## Saving data

For each `read_**` function to load data, there is a `to_**` method attached to `Series` and `DataFrames`.

#### Excel spreadsheets

Uncomment the following code cell and run to save the whole `DataFrame` to an Excel file.

In [ ]:
# with pd.ExcelWriter("test.xls") as our_writer:
#     air_quality.to_excel(writer, sheet_name='Blah-blah')

#### Exercise: writing to CSV text files

Find a method to save `DataFrames` to a text file (or whatever format you like more).

In [ ]:
# Your code here

## Some statistics

In [ ]:
# air_quality.describe()

## Computing correlations

Both `Series` and `DataFrames` have a **`corr()`** method to compute the correlation coefficient.

In [ ]:
air_quality.NOx.corr(air_quality['CO'])

If series are already grouped into a `DataFrame`, computing all correlation coefficients is trivial:

In [ ]:
air_quality.corr()

If you want to visualise this correlation matrix, uncomment the following code cell.

In [ ]:
# fig, ax = plt.subplots()
# p = ax.imshow(air_quality.corr(), interpolation="nearest", cmap='RdBu_r', vmin=-1, vmax=1)
# ax.set_xticks(np.arange(len(air_quality.corr().columns)))
# ax.set_yticks(np.arange(len(air_quality.corr().index)))
# ax.set_xticklabels(air_quality.corr().columns)
# ax.set_yticklabels(air_quality.corr().index)
# fig.colorbar(p)

## Creating DataFrames

* `DataFrame` can also be created manually, by grouping several `Series` together.
* Now just for fun we switch to **another dataset**
    - create 2 `Series` objects from 2 CSV files
    - create a `DataFrame` by combining the two `Series`

* Data are monthly values of
    - Southern Oscillation Index (SOI)
    - Outgoing Longwave Radiation (OLR), which is a proxy for convective precipitation in the western equatorial Pacific
* Data were downloaded from NOAA's website: https://www.ncdc.noaa.gov/teleconnections/

In [ ]:
soi_df = pd.read_csv('../data/soi.csv', skiprows=1, parse_dates=[0], index_col=0, na_values=-999.9,
                     date_parser=lambda x: pd.datetime.strptime(x, '%Y%m'))

In [ ]:
olr_df = pd.read_csv('../data/olr.csv', skiprows=1, parse_dates=[0], index_col=0, na_values=-999.9,
                     date_parser=lambda x: pd.datetime.strptime(x, '%Y%m'))

In [ ]:
df = pd.DataFrame({'OLR': olr_df.Value,
                   'SOI': soi_df.Value})

In [ ]:
# df.describe()

## Ordinary Least Square (OLS) regressions

### Primitive way: using numpy's polynomial fitting

In [ ]:
from numpy.polynomial import polynomial as P

In [ ]:
x = df['OLR'].values
y = df['SOI'].values

In [ ]:
idx = np.isfinite(x) & np.isfinite(y)

In [ ]:
coefs, stats = P.polyfit(x[idx], y[idx], 1, full=True)

In [ ]:
y2 = P.polyval(x, coefs)

In [ ]:
plt.plot(x, y, linestyle='', marker='o')
plt.plot(x, y2)

### Recommended (and more convenient) ways (require additional packages)

##### Statsmodels

In [ ]:
# import statsmodels.formula.api as sm

In [ ]:
# sm_model = sm.ols(formula="SOI ~ OLR", data=df).fit()

In [ ]:
# df['SOI'].plot()
# df['OLR'].plot()
# ax = sm_model.fittedvalues.plot(label="model prediction")
# ax.legend(loc="lower center", ncol=3)

More examples: https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html

### Exercise: create a scatter plot

* You can use `df.plot` function with the appropriate keywords
* What happens if you use "c=" keyword?
* Pass another parameter: `edgecolors='none'`

In [ ]:
# your code here

### Advanced scatter plot

Using the power of matplotlib, we can create a scatter plot with points coloured by the date index. To do this we need to import one additional submodule:

In [ ]:
# import matplotlib.dates as mdates

Convert `numpy.datetime64` objects (which are the indices of our `DataFrame`) to matplotlib floating point numbers. These numbers represent the number of days (fraction part represents hours, minutes, seconds) since 0001-01-01 00:00:00 UTC (assuming Gregorian calendar).

In [ ]:
# mdt = mdates.date2num(df.index)

Append the new data to the original `DataFrame`:

In [ ]:
# df['mpl_date'] = mdt

Create a scatter plot

In [ ]:
# ax = df.plot(kind='scatter', x='OLR', y='SOI', c='mpl_date',
#              colormap='viridis', colorbar=False, edgecolors='none')
# plt.colorbar(ax.collections[0], ticks=mdates.YearLocator(5), 
#              format=mdates.DateFormatter('%Y'))

## Exercise: rolling functions

**1. Subset data**

* Start by subsetting the SOI `DataFrame`
* Use either numerical indices, or, even better, datetime indices

In [ ]:
# your code here

**2. Plot the subset data**

* You can create figure and axis using `matplotlib.pyplot`
* Or just use the `plot()` method of pandas `DataFrame`

In [ ]:
# your code here

**3. Explore what `rolling()` method is**

* What does this method return?

In [ ]:
# df.rolling?

In [ ]:
# your code here

**4. Plot the original series and the smoothed series**

In [ ]:
# your code here

## References
* https://github.com/jonathanrocher/pandas_tutorial
* https://github.com/koldunovn/python_for_geosciences
* http://pandas.pydata.org/pandas-docs/stable/index.html#module-pandas
* http://pandas.pydata.org/pandas-docs/stable/10min.html